In [30]:
import pandas as pd
patients_df = pd.read_json('frames.json')
patients_df.head()

,id,labels,turns,user_id,wizard_id
0,e2c0fc6c-2134-4891-8353-ef16d8412c9a,"{'userSurveyRating': 4.0, 'wizardSurveyTaskSuc...",[{'text': 'I'd like to book a trip to Atlantis...,U22HTHYNP,U21DKG18C
1,4a3bfa39-2c22-42c8-8694-32b4e34415e9,"{'userSurveyRating': 3.0, 'wizardSurveyTaskSuc...","[{'text': 'Hello, I am looking to book a vacat...",U21E41CQP,U21DMV0KA
2,6e67ed28-e94c-4fab-96b6-68569a92682f,"{'userSurveyRating': 2.0, 'wizardSurveyTaskSuc...",[{'text': 'Hello there i am looking to go on a...,U21RP4FCY,U21E0179B
3,5ae76e50-5b48-4166-9f6d-67aaabd7bcaa,"{'userSurveyRating': 5.0, 'wizardSurveyTaskSuc...",[{'text': 'Hi I'd like to go to Caprica from B...,U22HTHYNP,U21DKG18C
4,24603086-bb53-431e-a0d8-1dcc63518ba9,"{'userSurveyRating': 5.0, 'wizardSurveyTaskSuc...","[{'text': 'Hello, I am looking to book a trip ...",U21E41CQP,U21DMV0KA


In [31]:
data= pd.DataFrame(columns=['text','intent','dst','or','dep','ret','bud'])


# Named Entity Recognition model

the following entities has to be recognized:
- destination
- origin
- budget
- departure date
- return date

In [32]:
for i in range(patients_df.shape[0]):
    dst=''
    orr=''
    strt=''
    fin=''
    bud=''
    intent=''
    
    try:
        b=patients_df.turns[i][0]['labels']['acts'][1]['args']
    except:
        b=[]
    try:
        intent=patients_df.turns[i][0]['labels']['acts'][0]['name']
    except:
        intent=''
    for y in b:
       
        if y['key']=='dst_city':
            des= y['val']
        if y['key']=='or_city':
            orr= y['val']
        if y['key']=='str_date':
            strt= y['val']
        if y['key']=='end_date':
            fin= y['val']
        if y['key']=='budget':
            bud= y['val']
    data = data.append({'text':patients_df.turns[i][0]['text'],'intent':intent,
                       'dst': des,
                       'bud':bud,
                       'or':orr,
                       'dep':strt,
                        'ret':fin }, ignore_index=True)
    data.astype({'bud': 'object'})

### preparing the data 
- replace 2016 year by 2020/2021

In [33]:
import numpy as np
data['dep'].iloc[:600]=data['dep'].iloc[:600].map(lambda x:str(x).replace('2016','2020'))
data['text'].iloc[:600]=data['text'].iloc[:600].map(lambda x:str(x).replace('2016','2020'))
data['ret'].iloc[:600]=data['ret'].iloc[:600].map(lambda x:str(x).replace('2016','2020'))
data['dep'].iloc[600:]=data['dep'].iloc[600:].map(lambda x:str(x).replace('2016','2021'))
data['text'].iloc[600:]=data['text'].iloc[600:].map(lambda x:str(x).replace('2016','2021'))
data['ret'].iloc[600:]=data['ret'].iloc[600:].map(lambda x:str(x).replace('2016','2021'))
data=data.replace("",np.nan)


In [34]:
data.describe()

,text,intent,dst,or,dep,ret,bud
count,1369,1366,1369,585,272,151,170
unique,1329,2,219,221,157,118,94
top,hi,inform,-1,Kabul,August 27th,24th,-1
freq,10,1022,50,14,10,3,15


In [35]:
data.isna().sum() 

text         0
intent       3
dst          0
or         784
dep       1097
ret       1218
bud       1199
dtype: int64

In [36]:
data['text'].map(lambda x: len(x.split())).max()

74

In [37]:
data.head(10)

,text,intent,dst,or,dep,ret,bud
0,I'd like to book a trip to Atlantis from Capri...,inform,Atlantis,Caprica,"Saturday, August 13, 2020",NaN,1700
1,"Hello, I am looking to book a vacation from Go...",inform,Mos Eisley,Gotham City,NaN,NaN,2100
2,Hello there i am looking to go on a vacation w...,inform,Gotham City,NaN,NaN,NaN,NaN
3,"Hi I'd like to go to Caprica from Busan, betwe...",inform,Caprica,Busan,"Sunday August 21, 2020","Wednesday August 31, 2020",NaN
4,"Hello, I am looking to book a trip for 2 adult...",inform,Denver,Kochi,NaN,NaN,"$21,300"
5,"Hey, i Want to go to St. Louis on the 17th of ...",inform,St. Louis,NaN,17th of August,NaN,NaN
6,I'm looking for a trip to Gotham City leaving ...,inform,Gotham City,Kakariko Village,"Saturday, August 13, 2020",NaN,$2400 USD
7,"Hello, I would like to book a 2-week trip leav...",greeting,Gotham City,NaN,NaN,NaN,NaN
8,"Hello, I am planning to book a trip to pittsborgh",inform,pittsborgh,NaN,NaN,NaN,NaN
9,"Hi, I need to go to Mos Eisley for a wedding, ...",inform,Mos Eisley,NaN,"Saturday, August 13, 2020","Tuesday, August 16, 2020",$3700


In [38]:
import string
words=[]
tokens=[]
for index, row in data.iterrows():
    if row['intent']=="inform":
        to=[]
        text=row['text'].lower().translate(str.maketrans('', '', string.punctuation))
        li= text.split()
        
        for w in li:
            if w in map(lambda x:x.lower(),str(row['dst']).translate(str.maketrans('', '', string.punctuation)).split()):
                to.append(1)
            elif w in map(lambda x:x.lower(),str(row['or']).translate(str.maketrans('', '', string.punctuation)).split()):
                to.append(2)
            elif w in map(lambda x:x.lower(), str(row['dep']).translate(str.maketrans('', '', string.punctuation)).split()):
                to.append(3)
            elif w in map(lambda x:x.lower(),str(row['ret']).translate(str.maketrans('', '', string.punctuation)).split()):
                to.append(4)
            elif w==str(row['bud']).translate(str.maketrans('', '', string.punctuation)):
                to.append(5)
            else:
                to.append(0)
        words.append(li)
        tokens.append(to)

In [39]:
words1=[item for sublist in words for item in sublist]
words1= list(map(lambda x:x.lower(),str(words1).translate(str.maketrans('', '', string.punctuation)).split()))
words1.append("none")

In [40]:
#check length
pairs=[]
for i in range(len (words)):
    if len(words[i])!=len(tokens[i]):
        print(str(len(words[i]))+str(len(tokens[i])))
    else:
        sentense=[]
        for y in range(len(words[i])):
            item=(words[i][y],tokens[i][y])
            sentense.append(item)
    pairs.append(sentense)
                    
       

In [41]:
prep=[]
for i in words:
    arr=[]
    for y in i:
        arr.append(words1.index(y))
    if len(arr)<74:
        arr.extend([len(words1)-1]*(74-len(arr)))
    prep.append(arr)


In [42]:
for i in tokens:
    
        if len(i)<74:
            
            i.extend([0]*(74-len(i)))
    

In [43]:
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding,Bidirectional,TimeDistributed
from tensorflow.keras import Sequential

input = Input(shape=(74,))

    # Add Embedding layer
    
x=Embedding(len(words1)+1, 1000)(input)

    # Add bidirectional LSTM
x=Bidirectional(LSTM(units=200, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode = 'concat')(x)

    # Add LSTM
x=LSTM(units=100, return_sequences=True, dropout=0.5, recurrent_dropout=0.5)(x)

z=Dense(6, activation="softmax")
    # Add timeDistributed Layer
y=TimeDistributed(z)(x)

model = Model(inputs=input,outputs=y)
model.summary()

    # Compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 74)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 74, 1000)          18203000  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 74, 400)           1921600   
_________________________________________________________________
lstm_2 (LSTM)                (None, 74, 100)           200400    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 74, 6)             606       
Total params: 20,325,606
Trainable params: 20,325,606
Non-trainable params: 0
_________________________________________________________________


In [44]:
import multiprocessing
history = model.fit(np.array(prep), np.expand_dims(np.array(tokens), axis=2),
                    
                    epochs=50,
                    verbose=1,
                    validation_split=0.1,workers=multiprocessing.cpu_count())

C:\Users\lesh3\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 919 samples, validate on 103 samples
Epoch 1/50
919/919 [==============================] - 30s 33ms/step - loss: 0.3151 - accuracy: 0.9376 - val_loss: 0.1146 - val_accuracy: 0.9759
Epoch 2/50
919/919 [==============================] - 29s 32ms/step - loss: 0.1350 - accuracy: 0.9693 - val_loss: 0.0973 - val_accuracy: 0.9759
Epoch 3/50
919/919 [==============================] - 29s 32ms/step - loss: 0.1121 - accuracy: 0.9693 - val_loss: 0.0798 - val_accuracy: 0.9761
Epoch 4/50
919/919 [==============================] - 29s 32ms/step - loss: 0.0871 - accuracy: 0.9704 - val_loss: 0.0603 - val_accuracy: 0.9786
Epoch 5/50
919/919 [==============================] - 29s 32ms/step - loss: 0.0672 - accuracy: 0.9753 - val_loss: 0.0489 - val_accuracy: 0.9841
Epoch 6/50
919/919 [==============================] - 28s 31ms/step - loss: 0.0577 - accuracy: 0.9794 - val_loss: 0.0418 - val_accuracy: 0.9874
Epoch 7/50
919/919 [==============================] - 28s 30ms/step - loss: 0.0496 - accur

In [45]:
model_json = model.to_json()
with open("modelChatNer.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("modelChatNer.h5")

In [47]:
with open('dict.txt', 'w') as f:
    for item in words1:
        f.write("%s\n" % item)

In [ ]:
text_file = open("dict.txt", "r")
lines = text_file.read().split('\n')
lines

In [417]:
trial= "travel to new york from miami the first week of august 2020 and back in september 2020".lower().translate(str.maketrans('', '', string.punctuation)).split()
trial.extend(["none"]*(74-len(trial)))

ready=[]
for i in trial:
        ready.append(words1.index(i))

### Test NER model

In [418]:
f=model.predict(np.array([ready]))
ff=np.argmax(f, axis=2)
arr=[]
for i in range (len(trial)):
    if ff[0][i]!=0:
        arr.append({trial[i]:[ff[0][i], f[0][i][ff[0][i]]]})
        
       
print(arr)

[{'new': [1, 0.865208]}, {'york': [1, 0.997886]}, {'miami': [2, 0.9922672]}, {'first': [3, 0.9869876]}, {'week': [3, 0.48395985]}, {'of': [4, 0.9000157]}, {'august': [3, 0.9650315]}, {'2020': [3, 0.9926489]}, {'september': [4, 0.9807802]}, {'2020': [3, 0.98176575]}]


# Textual analysis ( intent recognition ) model
-adding extra data not releated to bookings to have equal binary target

In [4]:
tweets= pd.read_csv('twcs.csv')

In [5]:
tweets['text'].describe()

count             2811774
unique            2782618
top       @AirAsiaSupport
freq                  287
Name: text, dtype: object

In [6]:
char_list=['@','//']
res = [ele for ele in tweets['text'] if all(ch not in ele for ch in char_list)]

In [7]:
from langdetect import detect
import re
res1=[]
for i in res:
    try:
        if  i.encode(encoding='utf-8').decode('ascii'):
            res1.append(i)
    except UnicodeDecodeError:
        var=''
        

In [22]:
add=pd.DataFrame({'text':res1,'intent':0}).iloc[10000:11000]

In [23]:
add.head()

,text,intent
10000,Really waiting at this door for ups to pull up,0
10001,I only fly virgin airlines bc they cabins turn...,0
10002,McDonald's fountain coke is the best,0
10003,do i dare leave my bed rn to get a mcdouble,0
10004,Someone bring me a strawberry cream pie from M...,0


In [24]:
full=add.append( pd.DataFrame({'text':data['text'],'intent':data['intent']}),ignore_index = True) 

In [25]:
full['intent']=full['intent'].apply(lambda x:1 if x=="inform" else 0)


In [26]:
full.iloc[1345:].head(10)

,text,intent
1345,yo fam want to take my girl down to Punta Cana,1
1346,I want to go to Paris from Maceio with 6200$.,1
1347,Hi! I'm looking to go from Essen to San Juan b...,1
1348,"Hello, I'm looking to go from Essen to Mannheim",1
1349,hi. I want to go to Tofino from Burlington. pl...,0
1350,Hi there,0
1351,hello,0
1352,Hey,0
1353,"Hi there, I'm planning a family vacation and w...",1
1354,I dream of a better life and I want to go to S...,1


In [27]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y=train_test_split(full['text'], full['intent'], test_size=0.1, random_state=42)

In [28]:
from keras.preprocessing.text import Tokenizer

from keras.preprocessing.sequence import pad_sequences
# define Tokenizer with Vocab Size
tokenizer = Tokenizer(num_words=15000)
tokenizer.fit_on_texts(full['text'])
 
X = tokenizer.texts_to_sequences(full['text'])
X = pad_sequences(X)

from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y=train_test_split(X, full['intent'], test_size=0.1, random_state=42)

Using TensorFlow backend.


In [29]:
import pickle

# saving
with open('tokenizer1Chat.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [666]:
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
input = Input(shape=(75,))
x = Embedding(15000 + 1, 300)(input)
y =LSTM(64)(x)
y= Dense(256,name='FC1')(y)
y= Activation('relu')(y)
y= Dropout(0.5)(y)
z= Dense(1,name='out_layer')(y)
z= Activation('sigmoid')(z)
model = Model(inputs=input,outputs=z)
model.summary()

Model: "model_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_28 (InputLayer)        (None, 75)                0         
_________________________________________________________________
embedding_26 (Embedding)     (None, 75, 300)           4500300   
_________________________________________________________________
lstm_40 (LSTM)               (None, 64)                93440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_15 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257

In [692]:
import multiprocessing
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
 
history = model.fit(train_x, train_y,
                    
                    epochs=50,
                    verbose=1,
                    validation_split=0.1, workers=multiprocessing.cpu_count())

C:\Users\lesh3\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1918 samples, validate on 214 samples
Epoch 1/50
1918/1918 [==============================] - 15s 8ms/step - loss: 0.0193 - accuracy: 0.9937 - val_loss: 1.2298 - val_accuracy: 0.8271
Epoch 2/50
1918/1918 [==============================] - 16s 8ms/step - loss: 0.0187 - accuracy: 0.9953 - val_loss: 1.1687 - val_accuracy: 0.8271
Epoch 3/50
1918/1918 [==============================] - 16s 8ms/step - loss: 6.8786e-04 - accuracy: 1.0000 - val_loss: 1.2227 - val_accuracy: 0.8224
Epoch 4/50
1918/1918 [==============================] - 15s 8ms/step - loss: 2.5350e-04 - accuracy: 1.0000 - val_loss: 1.2351 - val_accuracy: 0.8318
Epoch 5/50
1918/1918 [==============================] - 15s 8ms/step - loss: 1.6100e-04 - accuracy: 1.0000 - val_loss: 1.2491 - val_accuracy: 0.8318
Epoch 6/50
1918/1918 [==============================] - 15s 8ms/step - loss: 1.4550e-04 - accuracy: 1.0000 - val_loss: 1.2515 - val_accuracy: 0.8411
Epoch 7/50
1918/1918 [==============================] - 15s 8ms/ste

## Testing the Model

In [596]:
model.evaluate(test_x,test_y)

237/237 [==============================] - 0s 711us/step


[1.3341485478204012, 0.8649789094924927]

In [695]:
def check(Z):
    
    Z = tokenizer.texts_to_sequences([Z])

    Z=pad_sequences(Z, maxlen=75, dtype='int32', value=0)



    f=model.predict(Z)
    
    print(round(f[0][0],4))

check("Wanna get to LA for a week on thursday")
check("Call doctor I'am sick")
check("Hi, how are you?")
check("What time is it now?")
check("Travel from moscow to wien")

1.0
0.0
0.0
0.0004
0.9999
